# Dekoratory

Ogólna idea - modyfikacja działania funkcji bez ingerowania w tę funkcję. Przykłady:
- dodanie mierzenia czasu
- dodanie logowania do pliku
- obsługa błędów

Często nie chcemy modyfikować kodu funkcji, chociażby dlatego, żeby nie zmniejszać czytelności. Albo jeśli pewne zmiany chcemy wprowadzić do wielu funkcji to zamiast poprawiać w każdej, lepiej napisać jedną modyfikację.

## Przykład 1 – dzielenie przez 0

In [ ]:
def divide(a, b):
    return a / b

In [ ]:
divide(6, 3)  # divide(6, 0) błąd

Funkcja spełnia swoje zadanie, ale nie ma obsługi błędów. W takim razie modyfikujemy jej ciało

In [ ]:
def divide(a, b):
    if b == 0:
        return None
    else:
        return a / b

In [ ]:
divide(6, 3)

Modyfikacja obsługuje błąd, ale wymaga zmiany w funkcji. Zamiast tego, obsługę błędów można obsłużyć w innej funkcji

In [ ]:
def divide(a, b):
    return a / b

Poniższe funkcja przyjmuje jako argument funkcję `divide` i zwraca jej zmodyfikowaną wersję.

In [ ]:
def modify_division_function_to_handle_division_by_zero(divide_function): 
    def wrapper(a, b):  # wrapper to zmodyfikowana wersja funkcji
        if b == 0:
            return None
        else:
            return divide_function(a, b)  # nie duplikujemy kodu z funkcji dzielącej
    
    # Skąd w tym miejscu wiadomo czym jest a i b?
    # Nie wiadomo. po prostu zwracamy funkcję, która będzie miała takie parametry
    
    return wrapper  # zwraca ulepszoną funkcję

In [ ]:
divide(6, 3)  # divide(6, 0) spowoduje błąd

In [ ]:
modify_division_function_to_handle_division_by_zero(divide)  # to zwraca wrapper

In [ ]:
modify_division_function_to_handle_division_by_zero(divide)(6, 3)

In [ ]:
modify_division_function_to_handle_division_by_zero(divide)(6, 0)

Funkcji divide jako takiej nigdy nie chcemy wywoływać bo jest podatna na błąd

---

In [ ]:
def handle_zero_division(divide_function):
    def wrapper(a, b):
        if b == 0:
            return None
        else:
            return divide_function(a, b)    
    
    return wrapper


@handle_zero_division
def divide(a, b):
    return a / b

In [ ]:
divide

In [ ]:
divide(6, 3)

In [ ]:
divide(6, 0)

Powyższy zapis jest tak naprawdę wywołaniem funkcji `handle_zero_division` do której przekazujemy `divide`. To zwraca wrapper, do którego następnie przekazujemy argumenty. To co zwróci wrapper jest finalnym wynikiem powyższej linii.

Wydaje się że jest to strzelanie z armaty do wróbla, ale teraz tego walidatora możemy użyć również do innych funkcji, a nie tylko do tej jednej.

In [ ]:
def handle_zero_division(func):    # func - zwykle tak piszemy
    def wrapper(*args, **kwargs):  # args, kwargs - jesteśmy w stanie obsłużyć sytuacje gdzie są inne argumenty
        if args[1] == 0:
            return None
        else:
            return func(*args, **kwargs)    
    
    return wrapper


@handle_zero_division
def divide(a, b):
    return a / b


@handle_zero_division
def modulo(a, b):
    return a % b

In [ ]:
divide(6, 4)

In [ ]:
modulo(6, 4)

## Przykład 2 – transformacja wartości zwracanej przez funkcję

In [ ]:
def return_argument(x):
    return x

In [ ]:
return_argument(10)

In [ ]:
def square(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** 2

    return wrapper

In [ ]:
@square
def return_argument(x):
    return x

In [ ]:
return_argument(4)

## Przykład 3 – bardziej abstrakcyjny, obrazuje ideę dekoratora

In [ ]:
def my_decorator(func):
    return str.upper



@my_decorator
def my_function(x):
    return x

Pytanie: co zwroci wywołanie funkcji?

In [ ]:
my_function("hello")

> **ZADANIA**

## Wiele dekoratorów – przykład 1

In [ ]:
import time


def validate_division(func):
    def wrapper(*args, **kwargs):
        if args[1] == 0:
            return None
        else:
            return func(*args, **kwargs)
    
    return wrapper


def measure_time(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        stop = time.perf_counter()
        print("Czas: ", stop - start)
        
        return result

    return wrapper


def pure_divide(a, b):
    return a / b


@measure_time
@validate_division
def divide(a, b):
    return a / b

In [ ]:
divide(10, 0)

In [ ]:
measure_time(validate_division(pure_divide))(10, 0)

---

In [ ]:
@validate_division
@measure_time
def divide(a, b):
    return a / b

In [ ]:
divide(10, 0)

In [ ]:
validate_division(measure_time(pure_divide))(10, 0)

Nie ma printowania czasu, ponieważ tutaj najpierw wywołujemy funkcję walidującą, a ona po sprawdzeniu czy b==0 od razu zwraca None zamiast wywoływać kolejne funkcje.

Kiedy b!=0, czas będzie printowany.

In [ ]:
divide(10, 1)

## Wiele dekoratorów – przykład 2

In [ ]:
def add_exclamation(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        result += "!"
        return result
    return wrapper


def add_quotation(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        result += "?"
        return result
    return wrapper


@add_exclamation
@add_quotation
def make_upper(text):
    return text.upper()

In [ ]:
make_upper("hello")

Wołamy add_exclamation, które woła add_quotation, które woła make_upper

> **ZADANIA**

## Parametryzacja dekoratora

In [ ]:
def round_result(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        rounded_result = round(result)
        return rounded_result

    return wrapper



@round_result
def calculate_average(a, b):
    return (a + b) / 2


calculate_average(1.242342, 2.23)

---

In [ ]:
def actual_rounding_decorator(param):
    def round_result(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            rounded_result = round(result, param)
            return rounded_result

        return wrapper
    return round_result


@actual_rounding_decorator(3)
def calculate_average(a, b):
    return (a + b) / 2


calculate_average(1.242342, 2.23)